<a href="https://colab.research.google.com/github/FranciscoAguiar/DataScienceProgramming/blob/master/XGBoost_Feedback.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [81]:
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split,RepeatedStratifiedKFold
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelBinarizer, LabelEncoder

In [84]:
dataset = pd.read_csv('combined.csv', sep=',')

In [64]:
dataset.dtypes

id           int64
feedback    object
classe       int64
dtype: object

In [85]:
dataset.head(5)

,id,feedback,classe
0,23,Ler outros textos sobre o assunto para lher da...,0
1,24,Procure ler mais; para escrever mais acerca do...,0
2,35,Texto bom; precisa apenas ler mais um pouco; p...,0
3,37,Ter cuidado com copias da internet.,0
4,68,Carssimo; o trabalho precisa de uma sistematiz...,1


In [89]:
dataset["feedback"] = le.fit_transform(dataset["feedback"].astype(str))

In [90]:
str_cols = dataset.columns[dataset.columns.str.contains('(?:feedback)')]
clfs = {c:LabelEncoder() for c in str_cols}

for col, clf in clfs.items():
    dataset[col] = clfs[col].fit_transform(dataset[col])

In [91]:
# split data into X and y
X = dataset.iloc[:, :-1]
Y = dataset.iloc[:, -1]

In [92]:
X.shape

(1000, 2)

In [93]:
Y.shape

(1000,)

In [94]:
X

,id,feedback
0,23,389
1,24,635
2,35,732
3,37,726
4,68,236
...,...,...
995,13809,648
996,13810,650
997,13811,713
998,13813,720


In [95]:
seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=123)


In [96]:
rkf = RepeatedStratifiedKFold(n_splits=5, n_repeats=1, random_state=54321) #DIVIDI OS DADOS NOS CONJUNTOS QUE SERÃO DE      TREINO E TESTE EM CADA RODADA DA VALIDAÇÃO CRUZZADA


In [97]:
        X_train, X_test = [X.iloc[i] for i in train_index], [X.iloc[j] for j in test_index]
        y_train, y_test = [Y.iloc[i] for i in train_index], [Y.iloc[j] for j in test_index]

        X_train_np = np.asarray(X_train)
        X_test_np = np.asarray(X_test)
        y_train_np = np.asarray(y_train)
        y_test_np = np.asarray(y_test)

       
       

In [98]:
model = XGBClassifier()
model.fit(X_train_np, y_train_np)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [99]:
print(model)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)


In [100]:
# make predictions for test data
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]


In [101]:
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))


Accuracy: 66.67%
